In [1]:
import pandas as pd 
import numpy as np
from chembl_webresource_client.new_client import new_client

In [2]:
path = '/Users/f.nasirian/Dropbox (CCNR)/Biology/99_Toolbox/data/'
act = pd.read_csv(path+'Bioactivities/data/out/bioactivities.csv')

In [3]:
#will use chemical formula for validation 
gooseberry = {'Geraniin': 'C41H28O27',
 'Kaempferol': 'C15H10O6',
 'Quercetin': 'C15H10O7', 
 'Pyrogallol': 'C6H6O3', 
 'Gallic acid': 'C7H6O5',
 'Citric acid': 'C6H8O7', 
 'Ascorbic acid': 'C6H8O6', 
 'Ellagic acid': 'C14H6O8',
 'Chebulic acid': 'C14H12O11', 
 'Chebulinic acid': 'C41H32O27', 
 'Corilagin': 'C27H22O18', 
 'Emblicanin B': 'C34H20O22', 
 'Emblicanin A': 'C34H22O22', 
 'Pedunculagin': 'C34H24O22'}

In [4]:
_df = pd.DataFrame(list(gooseberry.keys()), columns=['chemName'])

Check [this](https://github.com/chembl/chembl_webresource_client/blob/master/demo_wrc.ipynb) to learn how to query chemicals' data in ChEMBL. 

In [5]:
molecule = new_client.molecule

In [6]:
#use iexact as the lookup type to get case-insensitive exact match (map chemical name to chembl id)
def f(x):
    temp_dict = molecule.filter(pref_name__iexact=x).only(['molecule_chembl_id', 'molecule_properties'])[0]
    if temp_dict:
        if temp_dict['molecule_properties']['full_molformula'] == gooseberry[x]:
            return(temp_dict['molecule_chembl_id']) 
        else:
            return(np.nan)

_df['chemId'] = _df.chemName.apply(lambda x: f(x))

In [7]:
_df = _df[_df.chemId.notnull()]

In [8]:
#add targets
_df = _df.merge(act, on='chemId', how='left')
_df = _df[_df.GeneId.notnull()]

In [9]:
#chemical-target count
_df.groupby('chemName')['GeneId'].size()

chemName
Ascorbic acid     1
Corilagin         1
Ellagic acid     28
Gallic acid       6
Kaempferol       17
Pedunculagin      1
Pyrogallol        5
Quercetin        34
Name: GeneId, dtype: int64

In [10]:
#common targets
_df.GeneId.value_counts()

HSD17B10    4
EGFR        3
TDP1        3
CSNK2B      3
CSNK2A1     3
           ..
IP6K2       1
TP53        1
FLT3        1
PIK3CG      1
F10         1
Name: GeneId, Length: 67, dtype: int64

In [11]:
_df[_df.GeneId == 'HSD17B10']

,chemName,chemId,GeneId
40,Quercetin,CHEMBL50,HSD17B10
52,Pyrogallol,CHEMBL307145,HSD17B10
60,Gallic acid,CHEMBL288114,HSD17B10
68,Ellagic acid,CHEMBL6246,HSD17B10


In [12]:
#number of chemicals with target
_df.chemId.nunique()

8

In [14]:
_df.to_csv('chemical_target.csv', index=False)